# Predizione di indici di borsa tramite financial news sentiment analysis

Progetto per tesi

Studente: Gian Luca Nediani

E-mail: gianluca.nediani@studio.unibo.it

## Introduzione

A partire da quanto mostrato nel paper [Deep Learning for Event-Driven Stock Prediction](https://www.ijcai.org/Proceedings/15/Papers/329.pdf), l'obiettivo è sviluppare una rete neurale in grado di predire l'andamento del mercato azionario tramite metodi di sentiment analysis: valutando le news di carattere finanziario di un dato giorno si vuole predire se il giorno dopo il valore di un certo indice di borsa aumenterà o diminuirà. Come nel paper, l'indice di riferimento utilizzato è _S&P500_, un indice rappresentativo delle performance delle 500 aziende più quotate nella borsa statunitense.

Per comprendere il significato semantico delle news e fare valutazioni sull'andamento del mercato, gli autori del paper rappresentano le news finanziarie come degli eventi. In questo esperimento invece, si farà ricorso a un'architettura Transformer, l'attuale stato dell'arte nel _natural language processing_. Grazie all'encoder di questa architettura, sarà possibile generare degli embedding in grado di rappresentare in maniera ricca il significato semantico dei titoli di notizie finanziarie. Questi embedding saranno poi l'input per una rete neurale di classificazione.

Come nel paper originale per realizzare una predizione per un dato giorno vengono utilizzate news finanziarie dell'intero mese precedente, al fine di realizzare un classificatore binario in grado di predire il rialzo/ribasso del valore dell'indice di riferimento S&P 500. Un modello efficace in tale predizione binaria, permetterebbe proficui guadagni nel mercato azionario grazie a operazioni di _day trading_.

Dopo aver creato gli embedding ed averli sfruttati per costruire un dataset, quest'ultimo verrà utilizzato per addestrare e valutare una rete neurale di classificazione basata su layer convoluzionali e lineari. Il modello sarà valutato nella sezione conclusiva sia con metriche di performance che in un contesto simulativo di trading, al fine di verificarne l'effettiva efficacia in un caso d'uso pratico.

### Installazioni e import

In [ ]:
!pip install yfinance # retrieve financial data
!pip install transformers # library for pre-trained language models
!pip install talib-binary # financial indicators

In [ ]:
import pandas as pd 
import numpy as np # work efficiently with n-dimensional arrays
import torch # deep learning library
import datetime # work with date format
import sklearn # pre-processing functions
import talib as ta
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup, BertTokenizer, BertModel
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from collections import defaultdict
from urllib.request import urlopen
import cloudpickle as cp
import math

### Parametri dell'esperimento

Il periodo coperto dal dataset finanziario va dal 2 febbraio 2007 al 29 novembre 2013. Tramite il seguente iperparametro è possibile modificare la data che segna l'inizio del test set.

In [ ]:
# date string format: yyyy-mm-dd, latest date in dataset is 2013-11-28
SPLIT_DATE = "2013-01-01"

Se questo flag è impostato a _True_, gli encoding delle notizie vengono calcolati, se è impostato a _False_ vengono invece scaricati gli embedding pre-calcolati in precedenza

In [ ]:
COMPUTE_ENCODINGS = False

### Riproducibilità

L'intero esperimento è implementato in PyTorch. Per garantirne la riproducibilità, vengono adottate una serie di misure preliminari, come indicato nella [documentazione PyTorch sulla riproducibilità](https://pytorch.org/docs/stable/notes/randomness.html).

In [ ]:
import random
import os

seed = 0
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

Un certo grado di variabilità nelle performance del modello sarà comunque presente in quanto l'operazione di max pooling 3d che verrà utilizzata nella rete neurale [non ha in PyTorch implementazioni deterministiche](https://pytorch.org/docs/stable/generated/torch.use_deterministic_algorithms.html).

## Encoder BERT

Path dei pesi del modello transformer preaddestrato. Pesi BERT standard con testo lower cased.

In [ ]:
MODEL_PATH = 'bert-base-uncased'

Viene utilizzata la GPU fornita da Colab, in quanto il calcolo degli embedding e l'addestramento della rete neurale tramite CPU sarebbero troppo lenti.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
CUDA_LAUNCH_BLOCKING = "1"

Using cuda device


Per prima cosa viene caricato l'encoder di un modello Transformer pre-addestrato. Questo encoder è già addestrato e dunque in grado di calcolare gli embedding delle notizie finanziarie.

In [ ]:
encoder = BertModel.from_pretrained(MODEL_PATH, output_hidden_states=True).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Viene caricato anche il relativo tokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained(MODEL_PATH, truncation=True)

## Encoding del testo in embedding con attention

### Il dataset

Il dataset utilizzato in questo esperimento è ottenuto a partire da due dataset di news finanziarie, entrambi utilizzati nel paper [Deep Learning for Event-Driven Stock Prediction](https://www.ijcai.org/Proceedings/15/Papers/329.pdf). Essi racchiudono rispettivamente 450341 news di natura finanziaria provenienti dalla testata giornalistica _Bloomberg_ e 109110 news di natura finanziaria provenienti dalla testata giornalistica _Reuters_. Sulle orme del paper sopracitato, sono stati estratti soltanto i titoli delle news, in quanto considerati più significativi del corpo della notizia. Inoltre, siccome il modello sviluppato può processare un numero finito di informazioni, i titoli sono stati filtrati, mantenendo solo quelli che includano il nome di uno o più degli indici di borsa che compongono l'indice _S&P500_. 
Le operazioni preliminari appena descritte portano ad avere il seguente file CSV, che per ogni giorno del periodo preso in esame (2007-2013), unisce i titoli di Bloomberg e Reuters.

In [ ]:
import os.path
from urllib.request import urlretrieve

if not os.path.exists("financial_titles.csv"):
    urlretrieve("https://raw.githubusercontent.com/gned0/NLP_stock_prediction/main/financial_titles_v6.csv", "financial_titles.csv")

df = pd.read_csv('financial_titles.csv', delimiter=',')
df = df.drop('Unnamed: 0', 1)
df = df.dropna(axis=0)
df["ts"] = df["ts"].astype(str)
df["ts"] = df["ts"].apply(lambda x: datetime.date(int(x[:4]), int(x[4:6]), int(x[6:8]))) # datetime format to compare dates
df = df.set_index('ts')
df = df.sort_index()
df.reset_index(level=0, inplace=True)
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,ts,title
0,2007-01-01,"Apple posts options expenses, stands by CEO Jo..."
1,2007-01-02,Apple options probe spotlights ex-officials: p...
2,2007-01-03,GE completes $626 mln Thailand's BAY stake dea...
3,2007-01-04,"US STOCKS-Indexes end up as Intel lifts techs,..."
4,2007-01-05,Nasdaq says no decisions made about LSE stake*...
...,...,...
3212,2016-08-11,Peru detects fresh oil spill from decades-old ...
3213,2016-08-12,Gilead to get attorney fees in hepatitis C pat...
3214,2016-08-13,Wall St. ends little changed though Nasdaq hit...
3215,2016-08-15,NYSE sees double-digit Asian IPOs through 2017...


In [ ]:
THRESHOLDDATE = datetime.date(2013, 12, 1)
df = df[(df['ts'] < THRESHOLDDATE)]

In [ ]:
df

,ts,title
0,2007-01-01,"Apple posts options expenses, stands by CEO Jo..."
1,2007-01-02,Apple options probe spotlights ex-officials: p...
2,2007-01-03,GE completes $626 mln Thailand's BAY stake dea...
3,2007-01-04,"US STOCKS-Indexes end up as Intel lifts techs,..."
4,2007-01-05,Nasdaq says no decisions made about LSE stake*...
...,...,...
2336,2013-11-25,FAA to warn airlines of engine icing risk on B...
2337,2013-11-26,ADM makes investment commitment to Aus farmers...
2338,2013-11-27,Dish chairman may remain involved in bid for L...
2339,2013-11-28,"Taiwan stocks rise again, Apple suppliers jump..."


### Encoding

La lunghezza massima dei titoli di news concatenati per ogni giorno nel dataset è di 512 parole, il massimo supportato dall'architettura dell'encoder BERT.

In [ ]:
MAX_LEN = 512

Viene definito un generatore di embedding che riceve un input testuale e ne restituisce l'embedding. L'input è troncato a 512 parole nel caso sia più lungo, oppure portato a tale lunghezza tramite padding. L'output dell'encoder è un tensore 512x768, per ogni parola viene restituita dunque una rappresentazione a 768 dimensioni. Il tensore 512x768 viene poi trasformato in un tensore 1x768 da un layer di pooling.

In [ ]:
class EncodingGenerator():
  def __init__(self, encoder, tokenizer, max_len):
    self.encoder = encoder
    self.tokenizer = tokenizer
    self.max_len = max_len

  def tokenize(self, text):
    
    tok_out = self.tokenizer.encode_plus(text, add_special_tokens=True,
                                          max_length=self.max_len, 
                                          pad_to_max_length=True,
                                          return_attention_mask=True, 
                                          return_tensors="pt")

    return tok_out['input_ids'].to(device), tok_out['attention_mask'].to(device)

  def encode(self, text):

    ids, att_mask = self.tokenize(text)
    output = self.encoder(ids, att_mask).pooler_output
    return output # pooled 1x768 output

In [ ]:
embedding_generator = EncodingGenerator(encoder, tokenizer, MAX_LEN)

Per ogni entry viene generato il rispettivo encoding

In [ ]:
if COMPUTE_ENCODINGS:  # embeddings computed from scratch
  encodings = []
  for _, row in df.iterrows():
    titles = row.title.split('. ')
    day_encodings = []
    with torch.no_grad():
      for t in titles:
        day_encodings.append(embedding_generator.encode(t))
    tensor = torch.cat(day_encodings)
    encodings.append(torch.mean(tensor, dim=0))
    

else: # download of pre-computed embeddings
  encodings = cp.load(urlopen("https://github.com/gned0/NLP_stock_prediction/blob/main/pooled_mean_encodings.pickle?raw=true"))

In [ ]:
for i, e in enumerate(encodings):
  encodings[i] = e.cpu().detach().numpy()

In [ ]:
series = pd.Series(encodings)

In [ ]:
len(series)

2341

Gli embedding vengono aggiunti al dataframe

In [ ]:
df["encoding"] = series
df.rename(columns={'ts':'Date'}, inplace = True) # rename index
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Date,title,encoding
0,2007-01-01,"Apple posts options expenses, stands by CEO Jo...","[-0.9005322, -0.4893921, -0.7482412, 0.7761569..."
1,2007-01-02,Apple options probe spotlights ex-officials: p...,"[-0.83810675, -0.44807652, -0.7801435, 0.67675..."
2,2007-01-03,GE completes $626 mln Thailand's BAY stake dea...,"[-0.8717985, -0.5080216, -0.8692926, 0.7485271..."
3,2007-01-04,"US STOCKS-Indexes end up as Intel lifts techs,...","[-0.8352278, -0.4769462, -0.81125015, 0.677047..."
4,2007-01-05,Nasdaq says no decisions made about LSE stake*...,"[-0.8495539, -0.46308327, -0.8066598, 0.698230..."
...,...,...,...
2336,2013-11-25,FAA to warn airlines of engine icing risk on B...,"[-0.8479992, -0.47481805, -0.8599102, 0.733987..."
2337,2013-11-26,ADM makes investment commitment to Aus farmers...,"[-0.8550904, -0.46234897, -0.82907385, 0.71492..."
2338,2013-11-27,Dish chairman may remain involved in bid for L...,"[-0.84285367, -0.4860228, -0.88535964, 0.73053..."
2339,2013-11-28,"Taiwan stocks rise again, Apple suppliers jump...","[-0.8323906, -0.5006465, -0.9622431, 0.7601057..."


## Aggiunta dei dati finanziari al dataset

Ora è necessario ottenere le informazioni relative all'andamento della borsa, in particolare dell'indice S&P500. Tramite il pacchetto yfinance viene creato un dataframe con informazioni sull'andamento di tale titolo (label ^GSPC) nel periodo corrispondente a quello coperto dal dataset di news.

In [ ]:
import yfinance as yf

stock = yf.download("^GSPC", start="2007-01-01", end="2013-12-01")
stock.reset_index(inplace=True)
stock.dropna(inplace=True)
#stock.rename(columns={'Date':'ts'}, inplace = True) # rename index
stock.drop(labels=["Adj Close"], axis=1, inplace=True) # drop of non-relevant columns 
stock['Date'] = stock['Date'].astype(str).apply(lambda x: x.replace('-', '')) # format date
stock['Date'] = stock['Date'].apply(lambda x: datetime.date(int(x[:4]), int(x[4:6]), int(x[6:8]))) # datetime format to match dates
stock_eval = stock.copy(deep=True) # save df as it is for later gain evaluation
stock

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Volume
0,2007-01-03,1418.030029,1429.420044,1407.859985,1416.599976,3429160000
1,2007-01-04,1416.599976,1421.839966,1408.430054,1418.339966,3004460000
2,2007-01-05,1418.339966,1418.339966,1405.750000,1409.709961,2919400000
3,2007-01-08,1409.260010,1414.979980,1403.969971,1412.839966,2763340000
4,2007-01-09,1412.839966,1415.609985,1405.420044,1412.109985,3038380000
...,...,...,...,...,...,...
1736,2013-11-22,1797.209961,1804.839966,1794.699951,1804.760010,3055140000
1737,2013-11-25,1806.329956,1808.099976,1800.579956,1802.479980,2998540000
1738,2013-11-26,1802.869995,1808.420044,1800.770020,1802.750000,3427120000
1739,2013-11-27,1803.479980,1808.270020,1802.770020,1807.229980,2613590000


Viene aggiunta la feature delta, la differenza fra il prezzo di chiusura e quello di chiusura.

In [ ]:
stock["Delta"] = stock["Close"] - stock["Open"]

Per ottenere le etichette da usare per la classificazione delle giornate nel mercato azionario, viene creato un valore binario: 0 se in un dato giorno il valore dell'indice chiude in calo rispetto all'apertura e 1 se al contrario chiude in rialzo.

In [ ]:
def binarize(x):
  if x > 0:
    return 1
  return 0

In [ ]:
targets = stock['Close'] - stock['Open']
targets = targets.apply(binarize)
targets

0       0
1       1
2       0
3       1
4       0
       ..
1736    1
1737    0
1738    0
1739    1
1740    0
Length: 1741, dtype: int64

Normalizzazione dei dati finanziari

In [ ]:
#stock.iloc[:, 1:] = (stock.iloc[:, 1:] - stock.iloc[:, 1:].min())/(stock.iloc[:, 1:].max() - stock.iloc[:, 1:].min())
#stock

In [ ]:
stock.iloc[:, 1:] = (stock.iloc[:, 1:]-stock.iloc[:, 1:].mean())/(stock.iloc[:, 1:].std(ddof=0)) # standard scaling
stock

,Date,Open,High,Low,Close,Volume,Delta
0,2007-01-03,0.560828,0.575394,0.554056,0.552794,-0.614998,-0.108611
1,2007-01-04,0.554569,0.541987,0.556531,0.560402,-0.941167,0.092818
2,2007-01-05,0.562184,0.526561,0.544897,0.522670,-1.006493,-0.566108
3,2007-01-08,0.522445,0.511752,0.537170,0.536355,-1.126347,0.209733
4,2007-01-09,0.538113,0.514529,0.543465,0.533163,-0.915117,-0.064128
...,...,...,...,...,...,...,...
1736,2013-11-22,2.220346,2.229987,2.233271,2.249919,-0.902245,0.461999
1737,2013-11-25,2.260260,2.244355,2.258795,2.239950,-0.945714,-0.262377
1738,2013-11-26,2.245117,2.245766,2.259620,2.241130,-0.616565,-0.025368
1739,2013-11-27,2.247787,2.245105,2.268302,2.260718,-1.241355,0.220538


Viene ora effettuato il merge fra il DataFrame che contiene gli embedding e quello che contiene i dati finanziari e le etichette target. Per riassumere, gli input X forniti al classificatore sono gli embedding delle news finanziarie e i dati finanziari dei 30 giorni precedenti il giorno g della predizione, mentre l'etichetta Y è un valore binario pari a 1 se nel giorno g il valore dell'indice chiude in rialzo rispetto all'apertura, e pari a 0 viceversa. Nel caso di un rialzo si potra sfruttare una corretta predizione per acquistare il titolo all'apertura della giornata di trading per poi rivenderlo in chiusura e ottenere un profitto. In caso di ribasso si potra ottenere un profitto vendendo allo scoperto il titolo e ricomprandolo a fine giornata. Le performance effettive ottenute in un contesto di day trading dal modello verranno analizzate nella sezione conclusiva

In [ ]:
df = df.merge(stock, on='Date')
df = df.set_index('Date') # dataset sorting
df = df.sort_index() # dataset sorting
df.reset_index(level=0, inplace=True) # dataset sorting
df

,Date,title,encoding,Open,High,Low,Close,Volume,Delta
0,2007-01-03,GE completes $626 mln Thailand's BAY stake dea...,"[-0.8717985, -0.5080216, -0.8692926, 0.7485271...",0.560828,0.575394,0.554056,0.552794,-0.614998,-0.108611
1,2007-01-04,"US STOCKS-Indexes end up as Intel lifts techs,...","[-0.8352278, -0.4769462, -0.81125015, 0.677047...",0.554569,0.541987,0.556531,0.560402,-0.941167,0.092818
2,2007-01-05,Nasdaq says no decisions made about LSE stake*...,"[-0.8495539, -0.46308327, -0.8066598, 0.698230...",0.562184,0.526561,0.544897,0.522670,-1.006493,-0.566108
3,2007-01-08,Escala Group to be delisted from Nasdaq Jan 10...,"[-0.8316893, -0.40585136, -0.76412916, 0.63134...",0.522445,0.511752,0.537170,0.536355,-1.126347,0.209733
4,2007-01-09,"Chevron 4th-qtr liquid, natural gas production...","[-0.86060226, -0.45243385, -0.6987639, 0.68889...",0.538113,0.514529,0.543465,0.533163,-0.915117,-0.064128
...,...,...,...,...,...,...,...,...,...
1737,2013-11-22,Microsoft sells over a million Xbox Ones in un...,"[-0.85503674, -0.47476488, -0.82900304, 0.7227...",2.220346,2.229987,2.233271,2.249919,-0.902245,0.461999
1738,2013-11-25,FAA to warn airlines of engine icing risk on B...,"[-0.8479992, -0.47481805, -0.8599102, 0.733987...",2.260260,2.244355,2.258795,2.239950,-0.945714,-0.262377
1739,2013-11-26,ADM makes investment commitment to Aus farmers...,"[-0.8550904, -0.46234897, -0.82907385, 0.71492...",2.245117,2.245766,2.259620,2.241130,-0.616565,-0.025368
1740,2013-11-27,Dish chairman may remain involved in bid for L...,"[-0.84285367, -0.4860228, -0.88535964, 0.73053...",2.247787,2.245105,2.268302,2.260718,-1.241355,0.220538


In [ ]:
#df['Open'] = df.shift(periods=1)['Open']

Raggruppamento delle informazioni finanziarie di ciascuna entry in una singola colonna

In [ ]:
df["financial information"] = df[['Open', 'High', 'Low', 'Close', 'Volume', 'Delta']].values.tolist()
df.drop(labels=['High', 'Low', 'Close', 'Volume', 'Delta'], axis=1, inplace=True)

Aggiunta etichette

In [ ]:
df["target"] = targets

In [ ]:
df

,Date,title,encoding,Open,financial information,target
0,2007-01-03,GE completes $626 mln Thailand's BAY stake dea...,"[-0.8717985, -0.5080216, -0.8692926, 0.7485271...",0.560828,"[0.5608277485864144, 0.5753942887782401, 0.554...",0.0
1,2007-01-04,"US STOCKS-Indexes end up as Intel lifts techs,...","[-0.8352278, -0.4769462, -0.81125015, 0.677047...",0.554569,"[0.5545689796245776, 0.5419865125181605, 0.556...",1.0
2,2007-01-05,Nasdaq says no decisions made about LSE stake*...,"[-0.8495539, -0.46308327, -0.8066598, 0.698230...",0.562184,"[0.5621842158842466, 0.5265609172957791, 0.544...",0.0
3,2007-01-08,Escala Group to be delisted from Nasdaq Jan 10...,"[-0.8316893, -0.40585136, -0.76412916, 0.63134...",0.522445,"[0.5224449063077798, 0.5117524104425408, 0.537...",1.0
4,2007-01-09,"Chevron 4th-qtr liquid, natural gas production...","[-0.86060226, -0.45243385, -0.6987639, 0.68889...",0.538113,"[0.5381129316892385, 0.5145290391026521, 0.543...",0.0
...,...,...,...,...,...,...
1737,2013-11-22,Microsoft sells over a million Xbox Ones in un...,"[-0.85503674, -0.47476488, -0.82900304, 0.7227...",2.220346,"[2.2203455477800897, 2.2299873611387584, 2.233...",0.0
1738,2013-11-25,FAA to warn airlines of engine icing risk on B...,"[-0.8479992, -0.47481805, -0.8599102, 0.733987...",2.260260,"[2.2602600922024365, 2.244355244300342, 2.2587...",0.0
1739,2013-11-26,ADM makes investment commitment to Aus farmers...,"[-0.8550904, -0.46234897, -0.82907385, 0.71492...",2.245117,"[2.2451172371060384, 2.245765885716116, 2.2596...",1.0
1740,2013-11-27,Dish chairman may remain involved in bid for L...,"[-0.84285367, -0.4860228, -0.88535964, 0.73053...",2.247787,"[2.2477868972428117, 2.245104681177601, 2.2683...",0.0


## Creazione del dataset per rete neurale di classificazione

A partire dal dataframe ottenuto in precedenza, è necessario ottenere il dataset finale da utilizzare per l'addestramento e la valutazione della rete neurale di classificazione. Come spiegato in precedenza, il classificatore farà uso delle informazioni relative all'intero mese precedente alla giornata da predire. Ogni entry di tale dataset avrà le seguenti feature:

*   **Data del giorno usata come indice**
*   **Dati *testuali* a lungo termine** (embedding dei 30 giorni precedenti, matrice 30x768)
* **Dati *finanziari* a lungo termine** (informazioni su valori di apertura, chiusura, picco, minimo e delta nei 30 giorni precedenti, matrice 30x4)
* **Token del giorno precedente** per calcolare in fase di training gli embedding a breve termine e ottenere maggiori performance.

Sarà fondamentale fare sì che il classificatore non possa accedere a informazioni future rispetto al momento in cui dovrà predire l'andamento del mercato. Il valore di apertura del giorno stesso può essere utilizzato in quanto è proprio all'apertura della giornata di trading che il modello effettua la predizione.

In un caso d'uso reale, il modello utilizza per predire un rialzo/ribasso nel valore dell'indice di riferimento S&P 500 nel giorno _g_, tutte le informazioni che fanno riferimento ai giorni da _g-1_ a _g-30_. All'apertura della giornata azionaria la predizione binaria inidicherà se per l'indice è previsto un aumento o una diminuzione nel valore(differenza fra prezzo di chiusura e prezzo di apertura); in questo modo è possibile acquistare o vendere allo scoperto il titolo per ottenere un profitto alla chisura dei mercati.





Prima di costruire il dataset con gli embedding e i dati finanziari, ne viene costruito uno identico con le date relative a tali informazioni. Questo viene fatto per verificare la correttezza temporale del dataset, ovvero che al momento di ogni predizione il modello non possa disporre di informazioni successive.

In [ ]:
def verify_dates(dataframe, lookback):
    lookback = lookback + 1
    data_raw = dataframe.to_numpy()
    data = []
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    data = np.array(data)
    x = data[:, :-1, 0] # dates of input days   
    y = data[:, -1, 0] # date
    
    return [x, y]

Viene ora costruito il dataset vero e proprio su cui si eseguiranno addestramento e valutazione della rete neurale di classificazione.

In [ ]:
def build_dataset(df):

  final_df = pd.DataFrame({'Date': [], '30-day encodings': [], '30-day financial data': [], 'Open value': [], 'Target': []})
  for _, row in df.iterrows():
    prior_day = df[df["Date"] == row.Date - datetime.timedelta(1)]
    index = df.index[df["Date"] == row.Date].tolist()[0]
    if(len(prior_day) and index >= 30):
      titles = df.iloc[index-30:index]["encoding"].values.tolist()
      financial_data = df.iloc[index-30:index]["financial information"].values.tolist()
      open_value = row.Open


      entry = {'Date': row.Date, '30-day encodings': np.array(titles), '30-day financial data': np.array(financial_data), 'Open value': open_value, 'Target': row.target}
      final_df = final_df.append(entry, ignore_index=True)
  return final_df

In [ ]:
final_df = build_dataset(df)
final_df

,Date,30-day encodings,30-day financial data,Open value,Target
0,2007-02-15,"[[-0.8717985, -0.5080216, -0.8692926, 0.748527...","[[0.5608277485864144, 0.5753942887782401, 0.55...",0.723287,1.0
1,2007-02-16,"[[-0.8352278, -0.4769462, -0.81125015, 0.67704...","[[0.5545689796245776, 0.5419865125181605, 0.55...",0.730377,0.0
2,2007-02-21,"[[-0.8316893, -0.40585136, -0.76412916, 0.6313...","[[0.5224449063077798, 0.5117524104425408, 0.53...",0.742763,0.0
3,2007-02-22,"[[-0.86060226, -0.45243385, -0.6987639, 0.6888...","[[0.5381129316892385, 0.5145290391026521, 0.54...",0.732653,0.0
4,2007-02-23,"[[-0.86288255, -0.45067856, -0.67497903, 0.686...","[[0.5199937554757775, 0.5162038395325933, 0.54...",0.727970,0.0
...,...,...,...,...,...
1337,2013-11-20,"[[-0.840774, -0.45131668, -0.746046, 0.6825028...","[[1.606659226941794, 1.6025182421415707, 1.589...",2.186996,1.0
1338,2013-11-21,"[[-0.8501678, -0.46904427, -0.84191597, 0.7253...","[[1.6236842538736638, 1.7351346752863315, 1.65...",2.160430,1.0
1339,2013-11-22,"[[-0.82561904, -0.48238593, -0.88654524, 0.710...","[[1.7559010821003282, 1.7830857233813515, 1.77...",2.220346,0.0
1340,2013-11-26,"[[-0.85612255, -0.47760183, -0.8575107, 0.7347...","[[1.8350303000666597, 1.8189171989465658, 1.80...",2.245117,1.0


## Preprocessing

### Suddivisione del dataset

Il dataset è diviso temporalmente a seconda della data scelta a inizio notebook

In [ ]:
def split_dataset(dataframe, split_date):
    is_train = dataframe['Date']<datetime.date(int(split_date[:4]), 
                                               int(split_date[5:7]), 
                                               int(split_date[8:10]))
    df_train = dataframe[is_train]
    df_test = dataframe[~is_train]

    return df_train, df_test  
    
df_train, df_test = split_dataset(final_df, SPLIT_DATE)

### Creazione Dataset e DataLoader

Vengono definite le strutture del Dataset e del relativo DataLoader per darlo in pasto alla rete neurale. I dati in input alla rete neurale sono per ogni istanza:


*   **Data long**, concatenazione dei dati testuali e finanziari a lungo termine
*   **Data mid**, concatenazione dei dati testuali e finanziari a medio termine
*   **Data short**, concatenazione dei dati testuali e finanziari a breve termine, compreso il valore di apertura del giorno stesso



In [ ]:
class FinancialDataset(Dataset):
  def __init__(self, data):
    self.dates = data[..., 0]
    self.encodings = data[..., 1]
    self.financial_data = data[..., 2]
    self.open_values = data[..., 3]
    self.targets = data[..., 4]

  def __len__(self):
    return len(self.dates)

  def __getitem__(self, item):
    date = str(self.dates[item])
    encodings = self.encodings[item]
    financial_data = self.financial_data[item] 
    open_value = self.open_values[item]
    target = self.targets[item]
    data_long = np.concatenate([encodings, financial_data], axis=1)
    data_mid =  np.concatenate([encodings[-7:, :], financial_data[-7:, :]], axis=1)
    data_short =  np.concatenate([encodings[-1, :], financial_data[-1, :]], axis=0)    
    return {
      'date': date,
      'data_long': torch.tensor(data_long, dtype=torch.float),
      'data_mid': torch.tensor(data_mid, dtype=torch.float),
      'data_short': torch.tensor(data_short, dtype=torch.float),
      'open_value': torch.tensor(open_value, dtype=torch.float),
      'targets': torch.tensor(target, dtype=torch.float)
    }

In [ ]:
def create_data_loader(data, batch_size):
  ds = FinancialDataset(
    data=data,
  )
  return DataLoader(
    ds,
    batch_size=BATCH_SIZE,
    shuffle=False
  )

In [ ]:
BATCH_SIZE = 64

In [ ]:
train_data_loader = create_data_loader(df_train.to_numpy(), BATCH_SIZE)
test_data_loader = create_data_loader(df_test.to_numpy(), BATCH_SIZE)

## Rete neurale convoluzionale per classificazione

Viene qui definita la rete neurale di classificazione: è composta da due blocchi convoluzionali, uno per processare i dati a lungo termine (30 giorni prima) e una per quelli a breve termine (7 giorni prima). Nei blocchi convoluzionali viene eseguita una convoluzione, seguita da normalizzazione, funzione di attivazione Tanh e dropout, infine è posto un layer di pooling estrarre i dati più significativi.
L'output dei due blocchi convoluzionali sono due vettori 1x768, che vengono concatenati col vettore 1x768 dei dati a breve termine (1 giorno prima) e il valore in apertura. Si ottiene dunque un vettore a 2323 dimensioni, che viene passato al layer in output per eseguire la classificazione binaria.

In [ ]:
class Classifier(nn.Module):
  def __init__(self):
        super(Classifier, self).__init__()

        self.cnn_long = self.convolutional_block(c_in=1, c_out=8, dropout=0.3, kernel_size=(3, 1), stride=(3, 1))
        self.maxpool_long = nn.MaxPool3d(kernel_size=(8, 10, 1))

        self.cnn_mid = self.convolutional_block(c_in=1, c_out=8, dropout=0.3, kernel_size=(3, 1), stride=(3, 1), padding=(1, 0))
        self.maxpool_mid = nn.MaxPool3d(kernel_size=(8, 3, 1))

        self.out = nn.Linear(2323, 1) 

  def forward(self, data_long, data_mid, data_short, open_value):
        x = self.cnn_long(data_long)
        x = self.maxpool_long(x).squeeze(1)

        y = self.cnn_mid(data_mid)
        y = self.maxpool_mid(y).squeeze(1)
        concat = torch.cat([x.squeeze(1), y.squeeze(1), data_short, open_value.unsqueeze(1)], dim=1) # concat of long, mid and short data into single vector of shace 1x2317

        return self.out(concat)
  
  def convolutional_block(self, c_in, c_out, dropout, **kwargs):
        block = nn.Sequential(
            nn.Conv2d(in_channels=c_in, out_channels=c_out, **kwargs),
            nn.BatchNorm2d(num_features=c_out),
            nn.ReLU(),
            nn.Dropout2d(p=dropout)
        )
        return block

In [ ]:
model = Classifier().to(device)

## Addestramento e valutazione

Vengono definiti un ottimizzatore e una funzione d'errore. La funzione di errore utilizzata è _binary cross entropy_ in quanto si tratta di un problema di classificazione binaria. Viene utilizzata la versione _with logits_ in quanto gli output della rete neurale non passano per una funzione di attivazione.

In [ ]:
EPOCHS = 200
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.BCEWithLogitsLoss().to(device)

Secondo le indicazioni della [documentazione PyTorch](https://pytorch.org/docs/stable/optim.html), vengono definiti gli step per l'addestramento e la valutazione del modello. 

In [ ]:
def train_epoch(model, data_loader, loss_fn, optimizer, scheduler, n_examples, device):
  model = model.train()
  losses = []
  correct_predictions = 0
  step = 0
  for d in data_loader:
      step += 1
      optimizer.zero_grad() # clears previous gradients
      data_long = d["data_long"].unsqueeze(1).to(device)
      data_mid = d["data_mid"].unsqueeze(1).to(device)
      data_short = d["data_short"].to(device)
      open_value = d["open_value"].to(device)
      targets = d["targets"].to(device)
      outputs = model(data_long, data_mid, data_short, open_value)
      preds = outputs>0    
      loss = loss_fn(outputs, targets.unsqueeze(1)) # computes loss
      correct_predictions += torch.sum(torch.transpose(preds, 0, 1) == targets)
      losses.append(loss.item())
      loss.backward() 
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
      optimizer.step() # optimizer takes step based on gradients
      scheduler.step() 
  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  step = 0
  with torch.no_grad(): # gradient computation disabled for evalutaion
      for d in data_loader:
        step += 1
        data_long = d["data_long"].unsqueeze(1).to(device)
        data_mid = d["data_mid"].unsqueeze(1).to(device)
        data_short = d["data_short"].to(device)
        open_value = d["open_value"].to(device)
        targets = d["targets"].to(device)
        outputs = model(data_long, data_mid, data_short, open_value)
        preds = (outputs>0)    
        loss = loss_fn(outputs, targets.unsqueeze(1))
        correct_predictions += torch.sum(torch.transpose(preds, 0, 1) == targets)
        losses.append(loss.item())
  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
history = defaultdict(list)
best_acc = 0
for epoch in range(EPOCHS):
  
  print(f'Epoch {epoch + 1}/{EPOCHS}')
  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,
    loss_fn,
    optimizer,
    scheduler,
    len(df_train),
    device
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')
  
  val_acc, val_loss = eval_model(
    model,
    test_data_loader,
    loss_fn,
    device,
    len(df_test)
  )


  print(f'Val   loss {val_loss} accuracy {val_acc}')
  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)
  if float(val_acc) > float(best_acc):
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_acc = val_acc

Epoch 1/200
Train loss 0.8297453083490071 accuracy 0.5047372954349698
Val   loss 0.6778186162312826 accuracy 0.5966850828729282
Epoch 2/200
Train loss 0.7148721406334325 accuracy 0.49526270456503013
Val   loss 0.7486032247543335 accuracy 0.4033149171270718
Epoch 3/200
Train loss 0.7273983202482525 accuracy 0.5271317829457365
Val   loss 0.8211050828297933 accuracy 0.4033149171270718
Epoch 4/200
Train loss 0.7296877032832095 accuracy 0.5064599483204134
Val   loss 0.8862437208493551 accuracy 0.4033149171270718
Epoch 5/200
Train loss 0.8022500088340357 accuracy 0.49612403100775193
Val   loss 0.6962501605351766 accuracy 0.43646408839779005
Epoch 6/200
Train loss 0.7421632001274511 accuracy 0.5090439276485789
Val   loss 0.7257506648699442 accuracy 0.4033149171270718
Epoch 7/200
Train loss 0.7118759374869498 accuracy 0.524547803617571
Val   loss 0.7468772729237875 accuracy 0.4033149171270718
Epoch 8/200
Train loss 0.7094230808709797 accuracy 0.5176571920757967
Val   loss 0.7238587935765585 ac

##Conclusioni

### Valutazione delle performance

Vengono caricati i pesi relativi all'epoca con i risultati migliori in fase di addestramento

In [ ]:
WEIGHTS = 'best_model_state.bin'
model.load_state_dict(torch.load(WEIGHTS))

<All keys matched successfully>

Viene fatta una valutazione finale del modello con tali pesi, con anche una confusion matrix per meglio interpretare i risultati.

In [ ]:
# computes confusion matrix matches
def compute_matches(preds, targets):
  TP = FP = FN = TN = 0
  targets = targets>0
  
  preds = preds.detach().cpu().numpy()
  targets = targets.detach().cpu().numpy()
    
  for i in range(len(preds)):
      if(preds[i] and targets[i]):
          TP += 1
      elif(preds[i] and not targets[i]):
          FP += 1
      elif(not preds[i] and targets[i]):
          FN += 1
      else:
          TN += 1
          
  return TP, FP, FN, TN

In [ ]:
def final_model_evaluation(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  step = 0
  dates = []
  results = []
  dictionary = {
      "TP": 0,
      "FP": 0,
      "FN": 0,
      "TN": 0
  }
  with torch.no_grad(): # gradient computation disabled for evalutaion
      for d in data_loader:
        step += 1
        data_long = d["data_long"].unsqueeze(1).to(device)
        data_mid = d["data_mid"].unsqueeze(1).to(device)
        data_short = d["data_short"].to(device)
        open_value = d["open_value"].to(device)
        targets = d["targets"].to(device)
        outputs = model(data_long, data_mid, data_short, open_value)
        preds = (outputs>0)
        
        matches = compute_matches(preds, targets)
        dictionary["TP"] += matches[0]
        dictionary["FP"] += matches[1]
        dictionary["FN"] += matches[2]
        dictionary["TN"] += matches[3]    

        dates.append(d["date"])
        results.append((outputs>0).cpu().detach().numpy())
        loss = loss_fn(outputs, targets.unsqueeze(1))
        correct_predictions += torch.sum(torch.transpose(preds, 0, 1) == targets)
        losses.append(loss.item())
  return correct_predictions.double() / n_examples, np.mean(losses), dictionary, results, dates

La **confusion matrix** sottostante mostra i valori predetti nelle righe e i valori reali nelle colonne, ed è da interpretarsi nel seguente modo:


*   True positive, rialzo dell'indice predetto correttamente 
*   False positive, il modello ha predetto un rialzo dell'indice quando questo è invece diminuito
*   False negative, il modello ha predetto una diminuzione dell'indice quando questo è invece aumentato
*   True negative, ribasso dell'indice predetto correttamente



In [ ]:
  val_acc, val_loss, dictionary, results, dates = final_model_evaluation(
    model,
    test_data_loader,
    loss_fn,
    device,
    len(df_test)
  ) 
  
  print(f'Final model: loss {val_loss} accuracy {val_acc}')
  pd.DataFrame([["True positives: " + str(dictionary["TP"]), "False positives: " + str(dictionary["FP"])],
              ["False negatives: " + str(dictionary["FN"]), "True negatives: " + str(dictionary["TN"])]],
               index=["Predicted positive (1)", "Predicted negative (0)"], columns=["Actually positive (1)", "Actually negative(0)"])

Final model: loss 0.6814564069112142 accuracy 0.6077348066298343


,Actually positive (1),Actually negative(0)
Predicted positive (1),True positives: 85,False positives: 48
Predicted negative (0),False negatives: 23,True negatives: 25


Il paper originale utilizzava due metriche di performance : l'accuracy totale del modello e il coefficiente MCC, calcolato come: $$\frac{T P ·T N −F P ·F N}{\sqrt{(TP +FP)(TP +FN )(TN +FP)(TN +FN )}}$$
(punteggio più alto è migliore). 

In [ ]:
mcc_num = dictionary["TP"]*dictionary["TN"]-dictionary["FP"]*dictionary["FN"]
mcc_den = math.sqrt((dictionary["TP"]+dictionary["FP"])*(dictionary["TP"]+dictionary["FN"])*(dictionary["TN"]+dictionary["FP"])*(dictionary["TN"]+dictionary["FN"]))
print("MCC:", mcc_num/mcc_den)

MCC: 0.14391497782701282


### Calcolo del Return of investment

Il modello sviluppato ha ottenuto le performance sperate, tuttavia per verificarne l'utilità pratica è importante simularne l'utilizzo in un contesto di compravendita di titoli di borsa. 

La simulazione avviene nel periodo coperto dai dati del test set: quando il modello prevede per una giornata un rialzo dell'indice S&P 500, acquista il titolo all'apertura dei mercati e lo rivende a fine giornata; viceversa se il modello prevede un ribasso dell'indice, lo vende allo scoperto (naked short selling) a inizio giornata e lo ricompra a fine giornata.

In [ ]:
results_df = pd.DataFrame({'Date': [], 'Prediction': []}) # prediction for each day in test set
results_df["Date"] = np.concatenate(dates, axis = 0)
results_df["Prediction"] = np.concatenate(results, axis = 0)
results_df['Date'] = results_df['Date'].astype(str).apply(lambda x: x.replace('-', ''))
results_df['Date'] = results_df['Date'].apply(lambda x: datetime.date(int(x[:4]), int(x[4:6]), int(x[6:8]))) # datetime format to match dates
results_df


,Date,Prediction
0,2013-01-03,True
1,2013-01-04,True
2,2013-01-08,True
3,2013-01-09,True
4,2013-01-10,True
...,...,...
176,2013-11-20,False
177,2013-11-21,False
178,2013-11-22,False
179,2013-11-26,False


Viene calcolato il delta fra chiusura e apertura per quantificare i guadagni/perdite di ogni giorno di trading.

In [ ]:
stock_eval["Delta"] = stock_eval['Close'] - stock_eval['Open'] # daily gain/loss
gains_df = stock_eval[["Date", "Open", "Delta"]]
gains_df = gains_df.merge(results_df, on='Date')
gains_df

,Date,Open,Delta,Prediction
0,2013-01-03,1462.420044,-3.050049,True
1,2013-01-04,1459.369995,7.099976,True
2,2013-01-08,1461.890015,-4.739990,True
3,2013-01-09,1457.150024,3.869995,True
4,2013-01-10,1461.020020,11.099976,True
...,...,...,...,...
176,2013-11-20,1789.589966,-8.219971,False
177,2013-11-21,1783.520020,12.329956,False
178,2013-11-22,1797.209961,7.550049,False
179,2013-11-26,1802.869995,-0.119995,False


In [ ]:
# GM server perché ha predetto il giorno successivo
#gains_df['Prediction'] = gains_df.shift(periods=1)['Prediction']

Il guadagno _gain_ calcolato di seguito indica il guadagno assoluto in dollari realizzato dal modello nel periodo coperto dal test set: il modello opera ogni giorno acquistando o vendendo allo scoperto una unità del titolo S&P 500. Si noti che i guadagni assoluti realizzati dal modello aumenterebbero se venisse investito più denaro. Se si operasse per esempio ogni giorno con 10 unità del titolo, il guadagno sarebbe moltiplicato di 10 volte.

In [ ]:
gain = gains_df[gains_df["Prediction"] == True]["Delta"].sum() - gains_df[gains_df["Prediction"] == False]["Delta"].sum()
gain

300.70947265625

Poichè il guadagno dipende dalla quantità di denaro investito, una misura più indicativa della efficacia pratica del modello è il _ROI (return of investment)_. Il ROI non è altro che rapporto tra il guadagno e la cifra investita. In questo caso si considera la cifra investita come la media del valore di apertura di S&P 500 nel periodo preso in esame.

In [ ]:
return_of_investment = gain/gains_df["Open"].mean()
return_of_investment

0.18478949873804618